In [ ]:
from neo4j import GraphDatabase
import pandas as pd
import numpy as np
import time
import random
import sys
import subprocess

random.seed(time.time())

# Functions
Before we start, we need to define a function for running queries on the database. The function that Neo4j uses for this purpose is execute_query().  This function takes 3 inputs which are the query as a string, one or a list of parameters and the name of the database. In return, it gives three outputs which are records, summary and keys. Since we only need the records for this project, we define the following function in order to keep the code cleaner and easier to read.

In [ ]:
def run_query(query, parameters=None):
    records, summary, keys = driver.execute_query(query, parameters,database_="neo4j")
    return records

# Starting the database
To start the database, we need to run the following command into the CMD:  
```
>>start ... Path to Neo4j directory ...\bin\neo4j.bat console
```
This will start the database and also gives us a port to open the Neo4j browser which gives us access to the database.  
However, since we want to have every operations automatic, we can run the following code. We use the function Popen() from the subprocess library. Unlike the run() that we used for importing data to the database, Popen() will run the command in the background, meaning we can continue with the rest of our code, while if we use run() we cannot continue unless we close stop the process which leads to stopping the database. Therefore, we should use Popen() that allows us to start the database in the background.

In [ ]:
root = r"... Path to Neo4j directory ..."
bot = rf"{root}\bin\neo4j.bat"
command = f'start "" "{bot}" console'

result = subprocess.Popen(command, cwd=f"{root}\\bin", shell=True)

<br>
When the database is running, we have to make a bridge between our python script and the database. We do this by calling the driver() from GraphDatabase library. This function takes the URI and Authentication data to connect to the database. The authentication is a tuple of username and password that is set for the database.

In [ ]:
URI = "neo4j://127.0.0.1:7689"
AUTH = ("neo4j", "Your Password")

driver =  GraphDatabase.driver(URI, auth=AUTH)
driver.verify_connectivity()

# Re-converting propertie's data types:
Using the Admin tool to import data to the database comes with its own chalanges, Some of which we saw in the LoadDatabase file. The final chalange is when we start the database, we can see that the types of the data have been changed to a string with the value of "null". Therefore, we need to fix this issue before we can start querying the database.

## Customers
Starting with the Customer nodes, the solution is easy for almost every properties. We set the type of the CUSTOMER_ID and nb_terminals to Integer, and others to Float, by simply using toInteger() and toFloat() functions.
The only chalanging part is the available_terminals property, which is suppose to be a list of terminal ids. The current values are stored as "[np.int64(id1), np.int64(id2), ... , np.int64(idn)]". We first replace every character with an "" (which is basically removing those caracters) except the numbers using the following code:
```
> replace(replace(replace(replace(replace(c.available_terminals, '[', ''), ']', ''), 'np.int64(', ''), ')', ''), ' ', '') AS replaced
```
This will gives us "id1,id2,...,idn" which is named "replaced". Then, with the following command we can turn this string into a list of integers:
```
> SET c.available_terminals = [x IN split(replaced, ',') WHERE x IS NOT NULL AND x <> '' | toInteger(x)]
```
- split(replaced, ',') will split the string into a list of substrings using comma as a split point -> "id1", "id2", ... , "idn"
- x IN ... -> iterates over the substrings.
- WHERE x IS NOT NULL AND x <> '' -> filters out empty substrings and nulls.
- toInteger() -> converts the remaining substring into integers.

the result is a list of integer values that will be set in available_terminals property.  
Note that we use apoc.periodic.iterate() to run the convert on all nodes. This way, we make the process safer in prevent memory overflow.

In [ ]:
query = """
    CALL apoc.periodic.iterate(
        "MATCH (c:Customer) RETURN c",
        "SET c.CUSTOMER_ID = toInteger(c.CUSTOMER_ID),
             c.nb_terminals = toInteger(c.nb_terminals),
             c.x_customer_id = toFloat(c.x_customer_id),
             c.y_customer_id = toFloat(c.y_customer_id),
             c.mean_amount = toFloat(c.mean_amount),
             c.mean_nb_tx_per_day = toFloat(c.mean_nb_tx_per_day),
             c.std_amount = toFloat(c.std_amount)
        WITH c, replace(replace(replace(replace(replace(c.available_terminals, '[', ''), ']', ''), 'np.int64(', ''), ')', ''), ' ', '') AS replaced
        SET c.available_terminals = [x IN split(replaced, ',') WHERE x IS NOT NULL AND x <> '' | toInteger(x)]",
        {batchSize:1000, parallel:true}
    );
    
    
"""

%time results = run_query(query)

## Terminals
For the terminal nodes, we only have three properties to convert. A TERMINAL_ID which is suppose to be an integer, and two coordinates which must be converted to float.

In [ ]:
query = """
    CALL apoc.periodic.iterate(
        "MATCH (t:Terminal) RETURN t",
        "SET t.TERMINAL_ID = toInteger(t.TERMINAL_ID),
            t.x_terminal_id = toFloat(t.x_terminal_id),
            t.y_terminal_id = toFloat(t.y_terminal_id)",
        {batchSize:1000, parallel:true}
    );
"""

%time results = run_query(query)

## Transactions
Finally, we get to the Transaction nodes. Just like the other two, the process is simple for all properties. Only for TX_DATETIME we need to have a small configuration, which is replacing the space between date and time with the letter T.

In [ ]:
query = """
    CALL apoc.periodic.iterate(
        "MATCH (t:Transaction) RETURN t",
        "SET t.TRANSACTION_ID = toInteger(t.TRANSACTION_ID),
        t.TX_FRAUD_SCENARIO = toInteger(t.TX_FRAUD_SCENARIO),
        t.TX_TIME_DAYS = toInteger(t.TX_TIME_DAYS),
        t.TX_TIME_SECONDS = toInteger(t.TX_TIME_SECONDS),
        t.TERMINAL_ID = toInteger(t.TERMINAL_ID),
        t.CUSTOMER_ID = toInteger(t.CUSTOMER_ID),
        t.TX_FRAUD = toInteger(t.TX_FRAUD),
        t.TX_AMOUNT = toFloat(t.TX_AMOUNT),
        t.TX_DATETIME = datetime(replace(t.TX_DATETIME, ' ', 'T'))",
        {batchSize:1000, parallel:true}
    );
"""

%time results = run_query(query)

the runtimes for each one are as follow:  
- ~1s for Customer nodes.
- Less than 1s for Terminal nodes.
- ~7s, ~11s and ~22s for Database 1, 2 and 3, respectivelly.

## Adding a new relationship
In the end, we only need to add the new relationship that is derived from the existing two relationships. 

In [ ]:
query = """
    MATCH (c:Customer)-[:MADE]->(tx:Transaction)-[:OCCURRED_AT]->(t:Terminal)
    WITH c, t,
        count(tx) AS tx_count,
        sum(tx.TX_AMOUNT) AS sum_amount,
        avg(tx.TX_AMOUNT) AS mean_amount
    MERGE (c)-[r:USED]->(t)
    SET r.TX_COUNT = tx_count,
        r.SUM_AMOUNT = sum_amount,
        r.MEAN_AMOUNT = mean_amount
"""

%time results = run_query(query)

<br>
These were the final data processing before we can start with the operations. 

# Operations
Now that our database is ready, we can start with the operations.  
Note that all the codes here are the final and optimized versions. The original codes are discussed in the report.

## (a)
_For each customer X, identify the customer Y (or the costumers) that share at least 3 terminals in which Y executes transactions and the spending amount of Y differs less than the 10% with respect to that of X. Return the name of X, the spending amount of X, the spending amount of the related costumer Y and the spending amount of Y._  
<br>
With the help of our new relationship :USED, this operation can be queried very easily. We match two customers to a mutual terminal. we make sure that the id of first customer is smaller than the second customer. This makes sure that duplicates will not be considered in the results. Then we count the number of terminals that these customers have in common and filter out the records that have less than 3 mutual terminals. Finally, we check the required condition on the spending amount of both customers.  
<br>
We run the query using run_query(), and also calculate the runtime using %time. We then print the number of results and the list of results.

In [ ]:
query = """
    MATCH (X:Customer)-[:USED]->(t:Terminal)<-[:USED]-(Y:Customer)
    WHERE elementId(X) < elementId(Y)
    WITH X, Y, count(DISTINCT t) AS shared
    WHERE shared >= 3
      AND abs(X.mean_amount - Y.mean_amount) < X.mean_amount * 0.1
    RETURN X.CUSTOMER_ID AS customer_X, 
           X.mean_amount AS spending_amount_X, 
           Y.CUSTOMER_ID AS customer_Y, 
           Y.mean_amount spending_amount_Y
    ORDER BY X.CUSTOMER_ID
    """

%time results = run_query(query)
print(f"Number of records: {len(results)}")
for record in results:
    print(record.data())

## (b)
_For each terminal identify the possible fraudulent transactions of the current month. The fraudulent transactions are those whose import is higher than 20% of the average import of the transactions executed on the same terminal in the previous month._  
<br>
We can devide this query to three parts:  
- In part I, we find the latest month and compute orioer month boundaries.  
- In part II, we calculate the average transaction amount on each terminal for the previous month.  
- Finally in part III, we look for transactions which their transaction amount is more than 20% of the average treansaction amount of the previous month on the same terminal.

In [ ]:
query = """
   // Part I
MATCH (tr:Transaction)
WITH datetime.truncate('month', max(tr.TX_DATETIME)) AS latest_month
WITH latest_month,
     latest_month - duration('P1M') AS prev_month,
     latest_month + duration('P1M') AS next_month

// Part II
MATCH (prtr:Transaction)-[:OCCURRED_AT]->(t:Terminal)
WHERE prtr.TX_DATETIME >= prev_month AND prtr.TX_DATETIME < latest_month
WITH t, avg(prtr.TX_AMOUNT) AS avg_tr, latest_month, next_month

// Part III
MATCH (crtr:Transaction)-[:OCCURRED_AT]->(t)
WHERE crtr.TX_DATETIME >= latest_month AND crtr.TX_DATETIME < next_month
  AND crtr.TX_AMOUNT > avg_tr * 1.2

RETURN crtr.TRANSACTION_ID AS transaction_id,
       crtr.TX_AMOUNT AS transaction_amount,
       t.TERMINAL_ID AS terminal_id;
    """

%time results = run_query(query)
print(f"Number of records: {len(results)}")
for record in results:
    print(record.data())

## (c)
_Given a user u, determine the “co-customer-relationships CC of degree k”. A user u’ is a co customer of u if you can determine a chain “u1-t1-u2-t2-…tk-1-uk“ such that u1=u, uk=u’, and for  each 1<=I,j<=k, ui <> uj, and t1,..tk-1 are the terminals on which a transaction has been executed. Therefore, CCk(u)={u’| a chain exists between u and u’ of degree k}. Please, note that depending on the adopted model, the computation of CCk(u) could be quite complicated. Consider therefore at least the computation of CC3(u) (i.e. the co-costumer relationships of degree 3)_  
<br>
To break down the given definition:  
- u1: the first customer.
- uk: co-customer at degree k.
- k: the number of customers in the chain.
- k-1: the number of terminals in the chain.  

Knowing these information, we can find the chain for the co-customer at degree 3 which is:
start - t1 - c2 - t2 - c3
Having that, we can write our query starting with a MATCH with the path above. Then we need to make sure that there is no duplications in the customers. Meaning all the customers in the matched chains are different than another. In the end, we return the results.  
Notice that we used DISTINCT for the result. Because there might be more than one path between a customer and its co-customer at degree k. Since we only need to know the co-customer and not the number of paths between them, we use DISTINCT.

In [ ]:
query = """
        MATCH (start: Customer {CUSTOMER_ID: $startId})
        
        MATCH (start)-[:USED]->(:Terminal)<-[:USED]-(c2:Customer)
              -[:USED]->(:Terminal)<-[:USED]-(c3:Customer)
        
        WHERE start <> c2 AND 
              start <> c3 AND c2 <> c3
        RETURN DISTINCT c3.CUSTOMER_ID AS co_customer_degree_3
"""

# Choose your desired customer ID in integer
startId = 50

%time results = run_query(query, parameters={"startId": startId} )
print(f"Number of records: {len(results)}")
for record in results:
    print(record.data())

## (d)
### _Extend the logical model that you have stored in the NOSQL database by introducing the following information (pay attention that this operation should be done once the NOSQL database has been already loaded with the data extracted from the datasets):_

#### _i: Each transaction should be extended with:_

1. _The period of the day {morning, afternoon, evening, night} in which the 
transaction has been executed._
2. _The kind of products that have been bought through the transaction {high
tech, food, clothing, consumable, other}._
3. _The feeling of security expressed by the user. This is an integer value 
between 1 and 5 expressed by the user when conclude the transaction._

_The values can be chosen randomly._

This operation can be done nicely with Apoc plugin.  
First, we introduce two lists of period and product. then for every transaction, we first calculate a random float number r between 0 and 1. Then we use CASE to determine the selected period based on r. we set 3 numbers 20, 60 abd 90 to create the weights of 20, 40, 30 and 10 for 'morning', 'afternoon', 'evening' and 'night', respectively. This is added to make the distribution more realistic. For product type and security feel we make two other random number to choose between the list of products and a number in range of 1 to 5.

In [ ]:
period = ["morning", "afternoon", "evening", "night"]
product = ["high_tech", "food", "clothing", "consumable", "other"]

query = """
    CALL apoc.periodic.iterate(
        "MATCH (t:Transaction) RETURN t",
        "WITH t, rand() AS r
        SET t.PERIOD_DAY = 
                CASE
                    WHEN r < 0.20 THEN 'morning'
                    WHEN r < 0.60 THEN 'afternoon'
                    WHEN r < 0.90 THEN 'evening'
                    ELSE 'night'
                END,
             t.PRODUCT_TYPE = $products[toInteger(rand() * size($products))],
             t.SECURITY_FEEL = toInteger(rand() * 5) + 1",
         {batchSize:1000, parallel:true, params:{
             periods: $periods,
             products: $products
         }}
    )
"""

In [ ]:
%time results = run_query(query, parameters={"periods": period, "products": product})

### _ii: Customers that make more than three transactions  from the same terminal expressing a similar average feeling of security should be connected as “buying_friends”. Therefore also this kind of relationship should be explicitly stored in the NOSQL database and can be queried. Note, two average feelings of security are considered similar when their difference is lower than 1._
<br>
This operation can be easily done with the help of :USED. But unfortunately, we are missing one piece and that is the average feeling of the transactions a customer has made on a terminal. To do that, we first find all the transactions that a customer has made on one terminal, and store the average of SECURITY_FEEL property. Then we find the correspondong :USED relationship between this customer and terminal, and add the computed average feel to the relationship.

In [ ]:
query = """
    CALL apoc.periodic.iterate(
        "MATCH (c:Customer)-[:MADE]->(tx:Transaction)-[:OCCURRED_AT]->(t:Terminal)
        WITH c, t, avg(toFloat(tx.SECURITY_FEEL)) AS avg_feel
        RETURN c, t, avg_feel",
        "MATCH (c)-[u:USED]->(t)
        SET u.AVG_FEEL = avg_feel",
        {batchSize: 10000, parallel: true}
    )
"""
%time results = run_query(query)

Now, we have everything we need to create our new relationship. to do that, we first do a MATCH between two customers and a mutual terminal using :USED relationship. Then we make sure that:
- Customers are not the same.
- The number of transactions each customer has made on the terminal is more than 3.
- Both customers expressing a similar average feeling of security.

If all those conditions are met, we can connect the two customers with the new relationship.

In [ ]:
query = """
    MATCH (c1:Customer)-[u1:USED]->(t:Terminal)<-[u2:USED]-(c2:Customer)
    WHERE elementId(c1) < elementId(c2) AND
          u1.TX_COUNT > 3 AND u2.TX_COUNT > 3 AND
          abs(u1.AVG_FEEL - u2.AVG_FEEL) < 1
    MERGE (c1)-[:BUYING_FRIEND]->(c2)
"""

%time results = run_query(query)

## (e)
### _For each period of the day identifies the number of transactions that occurred in that period, and the average number of fraudulent transactions._
<br>
For the final operation, we simply group all the transactions by the period of day they have been made. We count the number of each group, count the number of fraudulent transactions and the average number of fraudulent transactions.

In [ ]:
query = """
    MATCH (t:Transaction)
    WITH t.PERIOD_DAY AS day, count(*) AS total_count, sum(t.TX_FRAUD) AS fraud_count, round(avg(t.TX_FRAUD)* 1000) / 1000 as avg_fraud
    RETURN day, total_count, fraud_count, avg_fraud
"""

%time results = run_query(query)
for record in results:
    print(record.data())